In [3]:
pip install torch torchxrayvision

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchxrayvision-1.2.3-py3-none-any.whl.metadata (18 kB)
  Using cached scikit_image-0.24.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached imageio-2.34.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached tifffile-2024.7.2-py3-none-any.whl.metadata (30 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 2.2 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 993.4 kB/s eta 0:00:0000:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 196.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 933.4 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to 

In [4]:
import os
import torch
import torchxrayvision as xrv
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np

/Users/apple/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/apple/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os
import torch
import torchxrayvision as xrv
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np

class KneeDataset(Dataset):
    def __init__(self, data_path, categories, img_size):
        self.data_path = data_path
        self.categories = categories
        self.img_size = img_size
        self.data = []
        self.labels = []
        self.label_dict = {category: i for i, category in enumerate(categories)}
        self._load_data()

    def _load_data(self):
        for category in self.categories:
            folder_path = os.path.join(self.data_path, category)
            img_names = os.listdir(folder_path)
            for img_name in img_names:
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    img = (img - 128) / 128 * 1024  # Normalize to [-1024, 1024]
                    self.data.append(img)
                    self.labels.append(self.label_dict[category])

        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        img = np.expand_dims(img, axis=0)  # Add channel dimension
        img = torch.tensor(img, dtype=torch.float32)
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)
        return img, label

data_path = '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/Training'
categories = ['1Doubtful', '4Severe', '2Mild', '0Normal', '3Moderate']
img_size = 299

dataset = KneeDataset(data_path, categories, img_size)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
import torch.nn as nn

# Load the DenseNet model from torchxrayvision
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.op_thresh = None  # This disables thresholding for binary classification

# Modify the first convolutional layer to accept single-channel input
first_conv_layer = list(model.features.children())[0]
if first_conv_layer.in_channels == 1:
    print("Model already configured for single-channel input.")
else:
    new_conv1 = nn.Conv2d(1, first_conv_layer.out_channels, kernel_size=7, stride=2, padding=3, bias=False)
    model.features[0] = new_conv1


Model already configured for single-channel input.


In [12]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')


Epoch 1/20, Loss: 2.8415, Accuracy: 0.1515
Epoch 2/20, Loss: 2.7639, Accuracy: 0.2424
Epoch 3/20, Loss: 2.6908, Accuracy: 0.2545
Epoch 4/20, Loss: 2.6276, Accuracy: 0.2727
Epoch 5/20, Loss: 2.5730, Accuracy: 0.3030
Epoch 6/20, Loss: 2.5294, Accuracy: 0.3152
Epoch 7/20, Loss: 2.4931, Accuracy: 0.3152
Epoch 8/20, Loss: 2.4624, Accuracy: 0.3333
Epoch 9/20, Loss: 2.4304, Accuracy: 0.3212
Epoch 10/20, Loss: 2.4022, Accuracy: 0.3879
Epoch 11/20, Loss: 2.3756, Accuracy: 0.4909
Epoch 12/20, Loss: 2.3479, Accuracy: 0.4909
Epoch 13/20, Loss: 2.3160, Accuracy: 0.5455
Epoch 14/20, Loss: 2.2635, Accuracy: 0.6485
Epoch 15/20, Loss: 2.2212, Accuracy: 0.5758
Epoch 16/20, Loss: 2.1982, Accuracy: 0.6485
Epoch 17/20, Loss: 2.1813, Accuracy: 0.6182
Epoch 18/20, Loss: 2.1679, Accuracy: 0.6848
Epoch 19/20, Loss: 2.1563, Accuracy: 0.6545
Epoch 20/20, Loss: 2.1493, Accuracy: 0.6485


Test

In [17]:
import os
import torch
import torchxrayvision as xrv
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np

class KneeDataset(Dataset):
    def __init__(self, data_path, categories, img_size=224):  # Default size to 224x224
        self.data_path = data_path
        self.categories = categories
        self.img_size = img_size
        self.data = []
        self.labels = []
        self.label_dict = {category: i for i, category in enumerate(categories)}
        self._load_data()

    def _load_data(self):
        for category in self.categories:
            folder_path = os.path.join(self.data_path, category)
            img_names = os.listdir(folder_path)
            for img_name in img_names:
                img_path = os.path.join(folder_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    img = (img - 128) / 128 * 1024  # Normalize to [-1024, 1024]
                    self.data.append(img)
                    self.labels.append(self.label_dict[category])

        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        img = np.expand_dims(img, axis=0)  # Add channel dimension
        img = torch.tensor(img, dtype=torch.float32)
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)
        return img, label

data_path = '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/Training'
categories = ['1Doubtful', '4Severe', '2Mild', '0Normal', '3Moderate']
img_size = 224  # Resize images to 224x224

dataset = KneeDataset(data_path, categories, img_size)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [19]:
# import torch.nn as nn

# # Load the DenseNet model from torchxrayvision
# model = xrv.models.DenseNet(weights="densenet121-res224-all")
# model.op_thresh = None  # This disables thresholding for binary classification

# # Modify the first convolutional layer to accept single-channel input
# first_conv_layer = list(model.model.children())[0]
# if first_conv_layer.in_channels == 1:
#     print("Model already configured for single-channel input.")
# else:
#     new_conv1 = nn.Conv2d(1, first_conv_layer.out_channels, kernel_size=7, stride=2, padding=3, bias=False)
#     model.model[0] = new_conv1
import torch.nn as nn

# Load the DenseNet model from torchxrayvision
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.op_thresh = None  # This disables thresholding for binary classification

# Modify the first convolutional layer to accept single-channel input
first_conv_layer = list(model.features.children())[0]
if first_conv_layer.in_channels == 1:
    print("Model already configured for single-channel input.")
else:
    new_conv1 = nn.Conv2d(1, first_conv_layer.out_channels, kernel_size=7, stride=2, padding=3, bias=False)
    model.features[0] = new_conv1


Model already configured for single-channel input.


In [20]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

num_epochs = 50
best_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, Best Accuracy: {best_accuracy:.4f}')


Epoch 1/50, Loss: 2.8778, Accuracy: 0.1152, Best Accuracy: 0.1152
Epoch 2/50, Loss: 2.8591, Accuracy: 0.1333, Best Accuracy: 0.1333
Epoch 3/50, Loss: 2.8448, Accuracy: 0.1576, Best Accuracy: 0.1576
Epoch 4/50, Loss: 2.8261, Accuracy: 0.1818, Best Accuracy: 0.1818


KeyboardInterrupt: 